## Get baseline refusal responses from DeepSeek on StrongREJECT Eval Dataset

In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from multiprocessing import Pool, Queue, Process, Manager
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
from tqdm import tqdm

# Configuration
hf_cache = "/scratch/etheridge/huggingface"
os.environ["HF_HOME"] = hf_cache

MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
DTYPE = "bfloat16"
MAX_TOKENS = 2048
SEED = 45

# Multi-GPU Configuration
GPUS = [1, 2, 3]  # Specify which GPUs to use
PROCESSES_PER_GPU = 4 # Number of processes per GPU
TOTAL_PROCESSES = len(GPUS) * PROCESSES_PER_GPU

IN_CSV = "../orthogonalized_outputs_2048.csv"
OUT_CSV = "results/baseline_multi_gpu.csv"


In [2]:
def initialize_model_on_device(gpu_id):
    """Initialize model on a specific GPU device"""
    device = f"cuda:{gpu_id}"
    print(f"Loading model on {device}...")
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=DTYPE,
        device_map=device,
    )
    model.eval()
    
    return tokenizer, model, device

def process_batch(args):
    """Process a batch of prompts on a specific GPU"""
    batch_data, gpu_id, process_id = args
    
    try:
        # Initialize model for this process
        tokenizer, model, device = initialize_model_on_device(gpu_id)
        
        results = []
        for idx, prompt in batch_data:
            try:
                # Tokenize input
                inputs = tokenizer(prompt, return_tensors="pt").to(device)
                
                # Generation parameters
                generation_kwargs = {
                    "max_length": MAX_TOKENS,
                    "do_sample": False,
                    "num_beams": 1,
                    "pad_token_id": tokenizer.eos_token_id,
                }
                
                # Generate response
                with torch.no_grad():
                    outputs = model.generate(inputs["input_ids"], **generation_kwargs)
                
                # Decode response
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                results.append((idx, response))
                
                # Clear GPU memory
                del inputs, outputs
                torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"Error processing row {idx} on GPU {gpu_id}: {str(e)}")
                results.append((idx, f"ERROR: {str(e)}"))
        
        return results
        
    except Exception as e:
        print(f"Error initializing GPU {gpu_id}, Process {process_id}: {str(e)}")
        return [(idx, f"INIT_ERROR: {str(e)}") for idx, _ in batch_data]

def create_batches(df, num_processes):
    """Split dataframe into batches for parallel processing"""
    indices = list(range(len(df)))
    batch_size = len(indices) // num_processes
    
    batches = []
    for i in range(num_processes):
        start_idx = i * batch_size
        if i == num_processes - 1:  # Last batch gets remaining items
            end_idx = len(indices)
        else:
            end_idx = (i + 1) * batch_size
        
        batch_indices = indices[start_idx:end_idx]
        batch_data = [(idx, df.iloc[idx]['forbidden_prompt']) for idx in batch_indices]
        
        # Assign GPU based on process number
        gpu_id = GPUS[i % len(GPUS)]
        process_id = i // len(GPUS)
        
        batches.append((batch_data, gpu_id, process_id))
    
    return batches

In [3]:

def main():
    print("Starting multi-GPU baseline evaluation...")
    print(f"Using GPUs: {GPUS}")
    print(f"Processes per GPU: {PROCESSES_PER_GPU}")
    print(f"Total processes: {TOTAL_PROCESSES}")
    
    # Load dataset
    print("Loading dataset...")
    df = pd.read_csv(IN_CSV)
    df = df[["forbidden_prompt"]].copy()
    df["response"] = None
    df["row_idx"] = df.index  # Add original row index
    
    print(f"Dataset loaded: {len(df)} samples")
    
    # Create output directory
    os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
    
    # Create batches for parallel processing
    batches = create_batches(df, TOTAL_PROCESSES)
    print(f"Created {len(batches)} batches")
    
    # Process batches in parallel with progress bar
    start_time = time.time()
    
    # Initialize progress bar
    pbar = tqdm(total=len(df), desc="Processing samples", unit="samples")
    
    with ProcessPoolExecutor(max_workers=TOTAL_PROCESSES) as executor:
        # Submit all batches
        future_to_batch = {executor.submit(process_batch, batch): i 
                          for i, batch in enumerate(batches)}
        
        # Collect results as they complete
        all_results = []
        completed_batches = 0
        processed_samples = 0
        
        for future in as_completed(future_to_batch):
            batch_idx = future_to_batch[future]
            try:
                batch_results = future.result()
                all_results.extend(batch_results)
                completed_batches += 1
                
                # Update progress bar
                batch_size = len(batch_results)
                processed_samples += batch_size
                pbar.update(batch_size)
                pbar.set_postfix({
                    'Batches': f"{completed_batches}/{len(batches)}",
                    'GPUs': f"{len(GPUS)}x{PROCESSES_PER_GPU}",
                    'Rate': f"{processed_samples/(time.time()-start_time):.1f} samples/s"
                })
                
                # Save intermediate results
                if completed_batches % 2 == 0 or completed_batches == len(batches):
                    save_intermediate_results(df, all_results, processed_samples)
                    
            except Exception as e:
                print(f"\nBatch {batch_idx} generated an exception: {e}")
                pbar.update(len(batches[batch_idx][0]))  # Update progress even on error
    
    pbar.close()
    
    # Update dataframe with results
    print("Updating dataframe with results...")
    for idx, response in all_results:
        df.at[idx, "response"] = response
    
    # Save final results
    print(f"Saving results to {OUT_CSV}...")
    df.to_csv(OUT_CSV, index=False)
    
    end_time = time.time()
    total_time = end_time - start_time
    
    print(f"\n{'='*60}")
    print(f"BASELINE EVALUATION COMPLETE")
    print(f"{'='*60}")
    print(f"Total time: {total_time:.2f} seconds ({total_time/60:.1f} minutes)")
    print(f"Average time per sample: {total_time/len(df):.2f} seconds")
    print(f"Processing rate: {len(df)/total_time:.1f} samples/second")
    print(f"Samples processed: {len(df)}")
    print(f"GPUs used: {len(GPUS)} (IDs: {GPUS})")
    print(f"Processes per GPU: {PROCESSES_PER_GPU}")
    print(f"Total processes: {TOTAL_PROCESSES}")
    print(f"Results saved to: {OUT_CSV}")
    print(f"{'='*60}")

def save_intermediate_results(df, results, processed_count):
    """Save intermediate results to prevent data loss"""
    temp_df = df.copy()
    for idx, response in results:
        temp_df.at[idx, "response"] = response
    
    temp_file = OUT_CSV.replace('.csv', '_temp.csv')
    temp_df.to_csv(temp_file, index=False)
    
    # Print status without disrupting progress bar
    tqdm.write(f"Checkpoint: {processed_count}/{len(df)} samples processed, saved to {temp_file}")

def monitor_gpu_usage():
    """Monitor GPU usage (optional utility function)"""
    import subprocess
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu,memory.used,memory.total', 
                               '--format=csv,noheader,nounits'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            for i, line in enumerate(lines):
                if i in GPUS:
                    util, mem_used, mem_total = line.split(', ')
                    print(f"GPU {i}: {util}% utilization, {mem_used}MB/{mem_total}MB memory")
    except Exception as e:
        print(f"Could not monitor GPU usage: {e}")


In [4]:
if __name__ == "__main__":
    # Optional: Monitor initial GPU state
    print("Initial GPU state:")
    monitor_gpu_usage()
    print()
    
    # Run main processing
    main()
    
    # Optional: Monitor final GPU state
    print("\nFinal GPU state:")
    monitor_gpu_usage()


Initial GPU state:
GPU 1: 0% utilization, 18133MB/95830MB memory
GPU 2: 0% utilization, 18651MB/95830MB memory
GPU 3: 0% utilization, 18835MB/95830MB memory

Starting multi-GPU baseline evaluation...
Using GPUs: [1, 2, 3]
Processes per GPU: 4
Total processes: 12
Loading dataset...
Dataset loaded: 95 samples
Created 12 batches


Processing samples:   0%|          | 0/95 [00:00<?, ?samples/s]

Loading model on cuda:1...Loading model on cuda:2...Loading model on cuda:1...Loading model on cuda:3...Loading model on cuda:2...Loading model on cuda:3...


Loading model on cuda:3...Loading model on cuda:1...Loading model on cuda:2...


Loading model on cuda:3...Loading model on cuda:1...
Loading model on cuda:2...






Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Checkpoint: 70/95 samples processed, saved to results/baseline_multi_gpu_temp.csv


Processing samples: 100%|██████████| 95/95 [09:13<00:00,  5.51s/samples, Batches=12/12, GPUs=3x4, Rate=0.2 samples/s]

Checkpoint: 95/95 samples processed, saved to results/baseline_multi_gpu_temp.csv


Processing samples: 100%|██████████| 95/95 [09:15<00:00,  5.84s/samples, Batches=12/12, GPUs=3x4, Rate=0.2 samples/s]

Updating dataframe with results...
Saving results to results/baseline_multi_gpu.csv...

BASELINE EVALUATION COMPLETE
Total time: 555.25 seconds (9.3 minutes)
Average time per sample: 5.84 seconds
Processing rate: 0.2 samples/second
Samples processed: 95
GPUs used: 3 (IDs: [1, 2, 3])
Processes per GPU: 4
Total processes: 12
Results saved to: results/baseline_multi_gpu.csv

Final GPU state:
GPU 1: 22% utilization, 18133MB/95830MB memory
GPU 2: 17% utilization, 18651MB/95830MB memory
GPU 3: 0% utilization, 18835MB/95830MB memory
